### Exercise A
Write a small function that computes the condition number of a matrix under the max norm 

$ cond_{inf}(M) = ||M||_{inf}||M^-1||_{inf} $

In [1]:
from numpy import array,append,linalg

def infNorm(matrix: array) -> float:
    '''returns the inf norm of the matrix'''
    return max(abs(matrix)) if len(matrix.shape) == 1 else max(sum(abs(matrix)))

def condInf(matrix: array) -> float:
    '''returns the condition number of a matrix under the max norm'''
    return infNorm(matrix)*infNorm(linalg.inv(matrix))

For three frequencies, $ \omega = {0.800, 1.146, 1.400} $ calculate the condition of numbers for the matrix $ E - \omega S $.
The right-hand-side z is given with 8 significant digits. How many significant digits could we guarantee in the solution of x if everything else were assumed exact? Why?

##### Solution:
We can calculate the values of the matrices E and S with the following code:

In [2]:
import numpy as np

Amat = 	np.array([
    [22.13831203, 0.16279204, 0.02353879, 0.02507880,-0.02243145,-0.02951967,-0.02401863],
    [0.16279204, 29.41831006, 0.02191543,-0.06341569, 0.02192010, 0.03284020, 0.03014052],
    [0.02353879,  0.02191543, 1.60947260,-0.01788177, 0.07075279, 0.03659182, 0.06105488],
    [0.02507880, -0.06341569,-0.01788177, 9.36187184,-0.07751218, 0.00541094,-0.10660903],
    [-0.02243145, 0.02192010, 0.07075279,-0.07751218, 0.71033323, 0.10958126, 0.12061597],
    [-0.02951967, 0.03284020, 0.03659182, 0.00541094, 0.10958126, 8.38326265, 0.06673979],
    [-0.02401863, 0.03014052, 0.06105488,-0.10660903, 0.12061597, 0.06673979, 1.15733569]]);

Bmat = np.array([
    [-0.03423002, 0.09822473,-0.00832308,-0.02524951,-0.00015116, 0.05321264, 0.01834117],
    [ 0.09822473,-0.51929354,-0.02050445, 0.10769768,-0.02394699,-0.04550922,-0.02907560],
    [-0.00832308,-0.02050445,-0.11285991, 0.04843759,-0.06732213,-0.08106876,-0.13042524],
    [-0.02524951, 0.10769768, 0.04843759,-0.10760461, 0.09008724, 0.05284246, 0.10728227],
    [-0.00015116,-0.02394699,-0.06732213, 0.09008724,-0.07596617,-0.02290627,-0.12421902],
    [ 0.05321264,-0.04550922,-0.08106876, 0.05284246,-0.02290627,-0.07399581,-0.07509467],
    [ 0.01834117,-0.02907560,-0.13042524, 0.10728227,-0.12421902,-0.07509467,-0.16777868]]);

yvec= np.array([-0.05677315,-0.00902581, 0.16002152, 0.07001784, 0.67801388,-0.10904168, 0.90505180]);

# constructs the E matrix
E = np.block([[Amat, Bmat],[Bmat, Amat]])
# constructs the S matrix
S = np.diag([1,1,1,1,1,1,1,-1,-1,-1,-1,-1,-1,-1])
# constructs the z vector
z = np.block([[yvec], [-yvec]])

The solutions for the three frequences are given by

In [3]:
omega = [0.800, 1.146, 1.400]

for f in omega: print(E-f*S)

[[ 2.13383120e+01  1.62792040e-01  2.35387900e-02  2.50788000e-02
  -2.24314500e-02 -2.95196700e-02 -2.40186300e-02 -3.42300200e-02
   9.82247300e-02 -8.32308000e-03 -2.52495100e-02 -1.51160000e-04
   5.32126400e-02  1.83411700e-02]
 [ 1.62792040e-01  2.86183101e+01  2.19154300e-02 -6.34156900e-02
   2.19201000e-02  3.28402000e-02  3.01405200e-02  9.82247300e-02
  -5.19293540e-01 -2.05044500e-02  1.07697680e-01 -2.39469900e-02
  -4.55092200e-02 -2.90756000e-02]
 [ 2.35387900e-02  2.19154300e-02  8.09472600e-01 -1.78817700e-02
   7.07527900e-02  3.65918200e-02  6.10548800e-02 -8.32308000e-03
  -2.05044500e-02 -1.12859910e-01  4.84375900e-02 -6.73221300e-02
  -8.10687600e-02 -1.30425240e-01]
 [ 2.50788000e-02 -6.34156900e-02 -1.78817700e-02  8.56187184e+00
  -7.75121800e-02  5.41094000e-03 -1.06609030e-01 -2.52495100e-02
   1.07697680e-01  4.84375900e-02 -1.07604610e-01  9.00872400e-02
   5.28424600e-02  1.07282270e-01]
 [-2.24314500e-02  2.19201000e-02  7.07527900e-02 -7.75121800e-02
  

Since some of the elements in the matrix A have more than 8 S.D. (Significant Digits) and the solution z is given with exactly 8 SD we can assume that the solution x should be provided with at least 8 SD to guarantee a solution with 8 SD

### Exercise B
For the three frequences calculate the relative forward error as follows: $ \frac{||\delta x||_{inf}}{||x||} \geq cond_{inf}(E - \omega S) \frac{||\delta \omega S ||}{|| E - \omega S ||}$ for the pertubation that the frequency $ \omega $ is changed by $ \delta \omega = \frac{1}{2} * 10^{-3} $. How many significant digits can we guarantee if omega is give with 3 digits after the comma?

##### Solution
The error is then given by the following code:

In [4]:
deltaOmega = 1e-3
getError = lambda freq: condInf(E- S.dot(freq)) * infNorm(S.dot(deltaOmega))/infNorm(E-S.dot(freq))

for i in omega:
    print(f"Error for frequency {i}:\t{round(getError(i)*100, 2)}%")

Error for frequency 0.8:	1.04%
Error for frequency 1.146:	481.01%
Error for frequency 1.4:	0.71%


Since omega is given with 3 or more SD we can assume that the guaranteed SD in the computed result are also 3 

### Exercise C
#### 1)
Write a function that takes as input a square matrix and returns a lower matrix L and an upper matrix U such that $ A = LU $

In [5]:
from numpy import array, inner, diag

A = array([
	[2, 4, 3, 5],
	[-4, -7, -5, -8],
	[6, 8, 2, 9],
	[4, 9, -2, 14]
	])

def LU_factorize(A):
    # uses pivot
	diagonal = A.diagonal()
	dim = len(A)
	L = diag([1 for _ in range(dim)])
	U = A 
	for col in range(dim):
		for row in range(1, dim - col)[::-1]:
			coeff = U[dim-row][col]/diagonal[col]
			L[dim-row][col] = coeff
			U[dim-row] = U[dim-row]-coeff*U[col]
			#print(dim - row, col)
	return L, U

dec =  LU_factorize(A)
print(f"LU factorization for A:\nL: {dec[0]}\n\nU: {dec[1]}")
print(f"L*U =\n{dec[0].dot([dec[1]])}")

LU factorization for A:
L: [[ 1  0  0  0]
 [-2  1  0  0]
 [ 3 -4  1  0]
 [ 2  1  3  1]]

U: [[ 2  4  3  5]
 [ 0  1  1  2]
 [ 0  0 -3  2]
 [ 0  0  0 -4]]
L*U =
[[[ 2  4  3  5]]

 [[-4 -7 -5 -8]]

 [[ 6  8  2  9]]

 [[ 4  9 -2 14]]]


#### 2)
Write a function that takes as input a lower triangular matrix (L) and a vector (b) and returns a column vector x such that $ Lx = b $ using the forward substitution algorithm 

In [6]:
L = array([
    [ 1,  0,  0],
    [ 4,  3,  0],
    [ 2,  2,  3]
])

b = array([3 ,1, 2])

def forward_substitute(L, z):
	x = array([]) 
	for m in range(len(L)):
		x = append(x, (z[m] - sum(L[m][i]*x[i] for i in range(m)))/L[m][m])
	return x

print(f"Forward substitution for L and b: {forward_substitute(L, b)}\nL times the Forward substitution: {L.dot(forward_substitute(L, b))}")

Forward substitution for L and b: [ 3.         -3.66666667  1.11111111]
L times the Forward substitution: [3. 1. 2.]


#### 3)
Write a function that takes as input an upper triangular matrix (U) and a vector (b) and returns a column vector x such that $ Ux = b $ using the backward substitution algorithm 

In [7]:
import numpy as np
U = array([
    [ 1,  2,  2],
    [ 0,  7,  5],
    [ 0,  0,  3]
])

b = array([3 ,5, 2])

def backward_substitute(L, z):
	x = []
	for m in range(len(L)):
		x.insert(0,(z[::-1][m] - sum(L[::-1][m][::-1][i]*x[i] for i in range(m)))/L[::-1][m][::-1][m])
	return array(x)
    
print(f"Backward substitution for U and b: {backward_substitute(U, b)}\nU times the backward substitution: {U.dot(backward_substitute(U, b))}")

Backward substitution for U and b: [1.19047619 0.23809524 0.66666667]
U times the backward substitution: [3. 5. 2.]


### Exercise D
Wrrite a function that calculates the frequency-dependent polarization $ \alpha(\omega) = Z^TX $. Then make a table of the polarizabilities for the three frequencies each with an error of $ \delta\omega = 1/2 * 10^{-3} $

In [14]:
def solve_alpha(omega):
    firstTerm = (E-omega*S)
    L, U = LU_factorize(firstTerm)
    x = backward_substitute(U, np.linalg.inv(L).dot(z))
    return z.transpose().dot(x)

print(solve_alpha(1))
# WORK IN PROGRESS....

ValueError: shapes (14,14) and (2,7) not aligned: 14 (dim 1) != 2 (dim 0)